# Introduction

# 🛒 Customer Churn Analysis Dashboard  

## Business Problem  
Customer churn is one of the biggest challenges for businesses. Losing customers not only decreases revenue but also increases acquisition costs.  

The goal of this analysis is to:
- Measure **churn rate** and **retention rate**
- Understand churn by **gender, age group, and revenue**
- Identify the **revenue impact of churn**
- Provide actionable **recommendations** to improve retention

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px #interactive data visualization
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as colors
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pio.templates.default='plotly_white'

In [ ]:
# Load the dataset
df=pd.read_csv('/kaggle/input/e-commerce-customer-for-behavior-analysis/ecommerce_customer_data_custom_ratios.csv')

In [ ]:
# Check first few rows
df.head()

In [ ]:
# Summary statistics
df.describe()

In [ ]:
# Basic dataset info
df.info()

In [ ]:
#Converting date column

df['Purchase Date']=pd.to_datetime(df['Purchase Date'],errors='coerce')
print(df['Purchase Date'].dtype)

In [ ]:
# Check missing values
df.isna().sum()

In [ ]:
df['Returns']=df['Returns'].fillna(0)
df['Returns'].isna().sum()

In [ ]:
print(df.drop_duplicates(inplace =True))

## 2. Identify and separate dataframe columns into categorical and numerical lists

In [ ]:
categorical_columns=[]
numerical_columns=[]
for i in df.columns:
    if (df[i].dtype=='object'):
       categorical_columns.append(i)
    else:
        numerical_columns.append(i)
print(f"There are {len(categorical_columns)} Categorical columns: {categorical_columns}")
print(f"There are {len(numerical_columns)} Numerical columns: {numerical_columns}")
print(f"Total number of columns :{len(numerical_columns)+len(categorical_columns)}")

In [ ]:
df['Purchase Month']=df['Purchase Date'].dt.month
df['Purchase year']=df['Purchase Date'].dt.year
df['Purchase week']=df['Purchase Date'].dt.dayofweek

## Monthly Sales Analysis

In [ ]:
sales_by_month=df.groupby('Purchase Month')['Total Purchase Amount'].sum().reset_index()
sales_by_month

fig=px.line(sales_by_month,
            x='Purchase Month',
            y='Total Purchase Amount',
            title='Monthly Sales Analysis')
fig.update_layout(title={
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top' },
    title_font=dict(
        family="Calibri",  # font style
        size=24,           # font size
        color="black"  # font color   
    )
                 )   
fig.show()

## Yearly Sales Analysis

In [ ]:
sales_by_year=df.groupby('Purchase year')['Total Purchase Amount'].sum().reset_index()

fig=px.line(sales_by_year,
            x='Purchase year',
            y='Total Purchase Amount',
            title='Yearly Sales Analysis')
fig.update_layout(title={
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top' },
    title_font=dict(
        family="Calibri",  # font style
        size=24,           # font size
        color="black"  # font color   
    )
                 )   
fig.show()

## Payment Method Distribution

In [ ]:
payment_counts = df['Payment Method'].value_counts().reset_index()
payment_counts.columns = ['Payment Method', 'Count']

# Create Plotly pie chart
fig = px.pie(
    payment_counts,
    names='Payment Method',   # labels around the pie
    values='Count',           # sizes of slices
    title='Payment Method Distribution',
    color_discrete_sequence=px.colors.qualitative.Bold,  # same palette as sns Set2
    hole=0, # 0 for normal pie, >0 for donut chart
height=500)

# Show percentages inside slices
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(title={
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top' },
    title_font=dict(
        family="Calibri",  # font style
        size=24,           # font size
        color="black"  # font color   
    )
                 )   

fig.show()

## Sales by Product Category

In [ ]:
sales_by_product_category = df.groupby('Product Category')['Total Purchase Amount'].sum().reset_index()

fig=px.bar(sales_by_product_category, 
           x="Product Category", 
           y="Total Purchase Amount", 
           title="Sales by Category",
          color='Product Category',
          color_discrete_sequence=px.colors.qualitative.Bold,
          width=700)
fig.update_layout(title={
        'text': 'Sales by Category',  # fancy text
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top' },
    title_font=dict(
        family="Calibri",  # font style
        size=24,           # font size
        color="crimson"  # font color   
    )
                 )   
fig.show()

## Avg. Sales by Gender

In [ ]:
avg_sales_by_gender=df.groupby('Gender')['Total Purchase Amount'].mean().reset_index()
fig=px.bar(avg_sales_by_gender,
           x='Gender',
           y='Total Purchase Amount',
           title='Avg. Sales by Gender',
           color='Gender',
          color_discrete_sequence=px.colors.qualitative.Bold)
fig.update_layout(title={
        'text': 'Avg. Sales by Gender',  # fancy text
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top' },
    title_font=dict(
        family="Calibri",  # font style
        size=24,               # font size
        color="crimson"        # font color
    )
                 )
fig.show()

## Product Category Demand Over the Years

In [ ]:
product_quantity_by_category = df.groupby(["Purchase year", "Product Category"], as_index=False)["Quantity"].sum()
fig = px.line(
    product_quantity_by_category,
    x="Purchase year",
    y="Quantity",
    color="Product Category",
    markers=True,
    title="Product Quantity Trends"
)

# Improve layout
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Total Quantity",
    legend_title="Product Category",
    hovermode="x unified"
)

# Show the figure
fig.show()

In [ ]:
product_category_returned = df.groupby('Product Category')['Returns'].sum().reset_index()
product_category_returned['Returns'] *=100
fig=px.bar(product_category_returned, 
           x="Product Category", 
           y="Returns", 
           title="Return Rate by Category (%)",
          color='Product Category',
          color_discrete_sequence=px.colors.qualitative.Bold)

fig.update_layout(title={
        'text': 'Return Rate by Product Category',  # fancy text
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top' },
    title_font=dict(
        family="Calibri",  # font style
        size=24,               # font size
        color="crimson"        # font color
    )
                 )
fig.show()

## Customer Age Distribution

In [ ]:
fig=px.histogram(df, 
                   x='Age', 
                 color="Product Category",
                   nbins=20,
                   title='Age Distribution',
                   labels={'Age': 'Age'}, 
                 height=500,
                 color_discrete_sequence=px.colors.qualitative.Bold)

fig.update_traces(texttemplate='%{y}', textposition='outside')
fig.update_layout(title={
        'text': 'Return Rate by Product Category',  # fancy text
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top' },
    title_font=dict(
        family="Calibri",  # font style
        size=24,               # font size
        color="crimson", # font color
    ),
    yaxis=dict(
    title='Number of People')
    
)
fig.show()

## Return Rate by Product Category

In [ ]:
return_rate = df.groupby('Product Category')['Returns'].sum().reset_index()
fig=px.bar(return_rate,
           x='Product Category',
           y='Returns',
           height=500,
           color='Product Category',
           color_discrete_sequence=px.colors.qualitative.Bold)
fig.update_traces(texttemplate='%{y}', textposition='outside')
fig.update_layout(
    title={
        'text': 'Return Rate by Product Category',  # fancy text
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(
        family="Calibri",  # font style
        size=24,               # font size
        color="crimson"        # font color
    )
)
fig.show()

## Top 10 Customers by Spend

In [ ]:
top_customers = df.groupby('Customer Name')['Total Purchase Amount'].sum().nlargest(10).reset_index()
fig=px.bar(top_customers,
           x='Customer Name',
           y='Total Purchase Amount',
           title='Top 10 Customers by Spend')
fig.update_traces(marker=dict(color="green"))

fig.update_layout(
    title={
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(
        family="Calibri",  # font style
        size=24,               # font size
        color="crimson"        # font color
    )
)
fig.show()

# Customer-Level Aggregation

## 👥 Aggregate Data at Customer Level

In [ ]:
customer_summary = df.groupby(["Customer ID", "Customer Name", "Gender", "Customer Age"], as_index=False).agg({
    "Total Purchase Amount": "sum",
    "Quantity": "sum",
    "Returns": "sum",
    "Churn": "max"   # if churn=1 at any time → customer is churned
})

customer_summary.rename(columns={
    "Total Purchase Amount": "Total_Spend",
    "Quantity": "Total_Quantity",
    "Returns": "Total_Returns",
    "Churn": "Churn_Flag"
}, inplace=True)

# Churn Metrics

## 📈 Key Churn Metrics

In [ ]:
total_customers = customer_summary.shape[0]
churned_customers = customer_summary[customer_summary["Churn_Flag"] == 1].shape[0]
active_customers = total_customers - churned_customers

churn_rate = (churned_customers / total_customers) * 100
retention_rate = 100 - churn_rate

lost_revenue = customer_summary.loc[customer_summary["Churn_Flag"] == 1, "Total_Spend"].sum()
retained_revenue = customer_summary.loc[customer_summary["Churn_Flag"] == 0, "Total_Spend"].sum()

print("Total Customers:", total_customers)
print("Churned Customers:", churned_customers)
print("Active Customers:", active_customers)
print(f"Churn Rate: {churn_rate:.2f}%")
print(f"Retention Rate: {retention_rate:.2f}%")
print("Revenue Lost due to Churn:", lost_revenue)
print("Revenue from Retained Customers:", retained_revenue)

## Segmentation by Gender & Age

In [ ]:
# 🧑‍🤝‍🧑 Churn by Gender
churn_by_gender = customer_summary.groupby("Gender")["Churn_Flag"].mean().reset_index()
churn_by_gender["Churn_Flag"] *= 100 

#🎂 Churn by Age Group
bins = [18, 25, 35, 45, 55, 65, 75]
labels = ["18-24", "25-34", "35-44", "45-54", "55-64", "65-70"]
customer_summary["Age_Group"] = pd.cut(df["Customer Age"], bins=bins, labels=labels, right=False)

churn_by_age = customer_summary.groupby("Age_Group")["Churn_Flag"].mean().reset_index()
churn_by_age["Churn_Flag"] *= 100

# Visualizations

In [ ]:
# 📊 Visualization 1: Active vs Churned
fig1 = px.pie(
    names=["Active", "Churned"],
    values=[active_customers, churned_customers],
    title="Customer Distribution: Active vs Churned",
    color_discrete_sequence=["blue", "orange"]
)
fig1.update_layout(
    title_text="Customer Distribution: Active vs Churned",
    title_x=0.5,    # centers horizontally
    title_y=0.95,   # pushes title slightly down
    title_xanchor="center",
    title_yanchor="top",
    title_font=dict(
        family="Calibri",
        size=24,
        color="black"
    )
)
fig1.show()

In [ ]:
# 📊 Visualization 2: Churn by Gender
fig2 = px.bar(
    churn_by_gender,
    x="Gender",
    y="Churn_Flag",
    title="Churn Rate by Gender",
    color="Gender",
    text=churn_by_gender["Churn_Flag"].round(2).astype(str) + '%', height=500
)
fig2.update_traces(textposition="outside")
fig2.update_layout(
    title={
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(
        family="Calibri",  # font style
        size=24,               # font size
        color="black"        # font color
    )
)
fig2.show()

In [ ]:
# 📊 Visualization 3: Churn by Age Group
fig3 = px.bar(
    churn_by_age,
    x="Age_Group",
    y="Churn_Flag",
    title="Churn Rate by Age Group",
    text=churn_by_age["Churn_Flag"].round(2).astype(str) + '%',
    color="Age_Group", height=500
)
fig3.update_traces(textposition="outside")
fig3.update_layout(
    title={
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(
        family="Calibri",  # font style
        size=24,               # font size
        color="black"        # font color
    )
)
fig3.show()

# 5.4 Revenue Impact
fig4 = go.Figure(data=[
    go.Bar(name="Revenue", x=["Retained", "Churned"], y=[retained_revenue, lost_revenue],
           marker_color=["green", "red"], text=[retained_revenue, lost_revenue], textposition="outside")
])
fig4.update_layout(title="Revenue Impact of Churn", height =500)
fig4.update_layout(
    title={
        'y':0.95,        # vertical position
        'x':0.5,         # horizontal center
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(
        family="Calibri",  # font style
        size=24,               # font size
        color="black"        # font color
    )
)
fig4.show()

# Final Insights & Recommendations

## 📌 Key Insights
- Churn Rate: ~30% of customers are leaving.
- Gender: Female customers churn slightly more than male customers.
- Age Group: Younger customers (18–24) show the highest churn rates.
- Revenue: A significant portion of revenue is concentrated in retained customers.

## 🎯 Recommendations
- Launch **loyalty programs** targeting younger customers.
- Personalize offers for high-value customers at risk of churn.
- Improve the **return process**, as returns may correlate with churn.
- Use predictive modeling to identify at-risk customers earlier.